# Birliktelik kuralları (Association Rules)

Birliktelik kuralları, market pazar analizlerinde ve tavsiye sistemlerinde kullanılan önemli makine öğrenme tekniklerinden biridir. Kullanıcı ve ürünler arasındaki ilişkinin incelenmesiyle kullanıcının alışkanlıkları gözlenmekte ve bu gözlem sonucunda çeşitli aksiyonlar uygulanmakta. Örneğin; Netflix'de önerilen filmler ve diziler, Amazon'da alışveriş sırasında tavsiye edilen ürünler, Spotify'de dinlenilen şarkılara göre oluşturulan daily mix listeleri gibi. Bunların hepsi satış, izlenme ve dinlenme gibi oranları arttırmak için yapılmaktadır.


Birliktelik kuralları için birçok algoritma kullanılmakta, bunların içerisinde en popüler olanı Apriori Algoritmasıdır.

Apriori algoritmasında bilmemiz gereken 3 önemli kavram var ; Support, Confidence ve Lift.

X ve Y markette bulunan ürünleri, N ise toplam ürün sayısını ifade etsin.

- Support(Destek): X ve Y'nin ne sıklıkta birlikte görüldüğünü belirtir. 

- Confidence(Güven): X ürününü alan bir müşterinin Y ürününü alma ihtimalini belirtir.

- Lift: X ürünü satın alındığında Y ürünün satışının ne kadar arttığını belirtir.

Birliktelik kuralı için minimum support ve minimum confidence değerleri belirlenir. Ürünlerin supportları bulunarak, minimum support değerinin altında kalanlar çıkartılır. Bu işlem ürünlerin ikişer üçer şekilde gruplanması ve minimum support değerine uymayanların çıkartılmasıyla devam eder.Bu işlemlerden sonra elde edilen kümenin confidence değeri hesaplanır, minimum confidence değerinin altında kalanların çıkartılır. Geriye kalanlarlar birliktelik kuralları oluşturulur.


In [5]:
# gerekli kütüphaneler import edildi, tüm satır ver sutunları görmek için ayarlar yapıldı.
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
# veri seti okutuldu, ilk beş gözleme bakıldı.
df = pd.read_csv("GroceryStoreDataSet.csv", names =['Products'])
df.head()

,Products
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"


In [9]:
# Veri setinin boyutu
df.shape

(20, 1)

In [11]:
# product sutunundaki veriler listeye atandı
data = list(df["Products"].apply(lambda x:x.split(',')))
data 

[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COCK'],
 ['BREAD', 'SUGER', 'BISCUIT'],
 ['COFFEE', 'SUGER', 'CORNFLAKES'],
 ['BREAD', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

In [17]:
# listeyi True ve false hale dönüştürme. One Hot Encoding işlemi yapılmıştır.
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_data = te.fit(data).transform(data)
df = pd.DataFrame(te_data,columns=te.columns_)
df.head()

,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,True,False,True,False,False,False,False,False,True,False,False
1,True,False,True,False,False,True,False,False,True,False,False
2,False,True,True,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,True,True,True,False,False
4,True,False,False,False,False,False,False,True,False,False,True


In [15]:
# Apriori en yaygın kullanılan fonk olarak da bilinmektedir. Support değeri için bir eşik değeri seçilir ve support değeri hesaplanır.
freq_items = apriori(df, min_support = 0.2, use_colnames = True, verbose = 1) 

Processing 42 combinations | Sampling itemset size 3


In [14]:
# bulunan support değerlerine erişilir.
freq_items

,support,itemsets
0,0.35,(BISCUIT)
1,0.20,(BOURNVITA)
2,0.65,(BREAD)
3,0.40,(COFFEE)
4,0.30,(CORNFLAKES)
5,0.25,(MAGGI)
6,0.25,(MILK)
7,0.30,(SUGER)
8,0.35,(TEA)
9,0.20,"(BISCUIT, BREAD)"


In [20]:
#association_rules fonk kullanılarak "confidence ve lift" değerlerimize erişiyoruz
rules = association_rules(freq_items, metric="confidence", min_threshold = 0.6)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(MAGGI),(TEA),0.25,0.35,0.2,0.800000,2.285714,0.1125,3.25
0,(MILK),(BREAD),0.25,0.65,0.2,0.800000,1.230769,0.0375,1.75
2,(CORNFLAKES),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80
3,(SUGER),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80
1,(SUGER),(BREAD),0.30,0.65,0.2,0.666667,1.025641,0.0050,1.05


- (MAGGI) ve (TEA)'in tüm alışverişler içerisinde birlikte görülme oranı 0.2'dir. (MAGGI) alan müşterilerin %80'i (TEA) de almaktadır. (MAGGI) olan alışverişlerde (TEA)'in satışı 2.28 artmaktadır.
- (MILK) ve (BREAD)'in tüm alışverişler içerisinde birlikte görülme oranı 0.2'dir. (MILK) alan müşterilerin %80'i (BREAD) de almaktadır. (MILK) olan alışverişlerde (BREAD)'in satışı 1.23 artmaktadır.
- (SUGER) ve (BREAD)'in tüm alışverişler içerisinde birlikte görülme oranı 0.2'dir. (SUGAR) alan müşterilerin %67'si (BREAD) de almaktadır. (SUGAR) olan alışverişlerde (BREAD)'in satışı 1.03 artmaktadır.
- (CORNFLAKES) ve (COFFEE)'in tüm alışverişler içerisinde birlikte görülme oranı 0.2'dir. (CORNFLAKES) alan müşterilerin %67'si (COFFEE) de almaktadır. (CORNFLAKES) olan alışverişlerde - (COFFEE)'in satışı 1.67 artmaktadır.
- (SUGER) ve (COFFEE)'in tüm alışverişler içerisinde birlikte görülme oranı 0.2'dir. (SUGAR) alan müşterilerin %67'si (COFFEE) de almaktadır. (SUGAR) olan alışverişlerde (COFFEE)'in satışı 1.67 artmaktadır.